In [1]:
# !wget https://huggingface.co/datasets/radenmuaz/masabih.org/resolve/main/mishkat_al_masabih.jsonl
# !wget https://huggingface.co/datasets/hazmannaim/umno-online.my/resolve/main/article_data.json
# !wget https://huggingface.co/datasets/rulkimi/tamil-oneindia/resolve/main/tamil-oneindia.json
# !wget https://huggingface.co/datasets/izardy/malaysia-elelong/resolve/main/train.csv

# !wget https://huggingface.co/datasets/Ammar-Azman/crawl-lirik-lagu-dot-net/resolve/main/lirik_lagu_dotnet_1_1000.jsonl
# !wget https://huggingface.co/datasets/Ammar-Azman/crawl-lirik-lagu-dot-net/resolve/main/lirik_lagu_dotnet_1000_end.jsonl
# !wget https://huggingface.co/datasets/aazwan/upsi_ejournal_of_music/raw/main/journal_with_full_text.json
# !wget https://huggingface.co/datasets/haziqzikry/foodie-my-food/resolve/main/foodie-my-food.json
# !wget https://huggingface.co/datasets/aazwan/malaysian_journal_of_analytical_science/raw/main/data.json -O malaysian_journal_of_analytical_science.json
# !wget https://huggingface.co/datasets/fattahharith/malaysian-journal-of-computing/raw/main/mjoc_pdf_text_extracted.json
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/muftiwp.gov.my.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/dermatology.org.my.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/raw/main/e-mjm.org.jsonl
    
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/myjms.mohe.gov.my.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/pgm-my.org.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/newera.edu.my.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/msss.com.my.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/mps.org.my.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/morthoj.org.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/mjpath.org.my.jsonl
    
# !wget https://huggingface.co/datasets/syafie-nzm/crawl-jurnaldbp/resolve/main/jurnaldbp.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/mjpharm.org.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/myjgeosc.com.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/myjsustainagri.com.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/akademisains.gov.my.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/crossref-pdf.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/Kamus_Dewan_Bahasa_Edisi_Keempat_pdf.pdf
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/melayu-pdf.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/majcafe.com.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/myjms.mohe.gov.my.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/newera.edu.my.jsonl
# !wget https://huggingface.co/datasets/hlmshkr/maljnutr-pdfs/resolve/main/maljnutr-pdf-extracted.json -O maljnutr.json
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/nutriweb.org.my.jsonl
# !wget https://huggingface.co/datasets/fattahharith/sejarah-stpm/raw/main/sejarahstpm-text-extracted.json
# !wget https://huggingface.co/datasets/fattahharith/malaysian-journal-of-computing/resolve/main/mjoc_pdf_text_extracted.json

In [2]:
from glob import glob
from tqdm import tqdm
import json
import os
import numpy as np
from transformers import AutoTokenizer
from streaming import MDSWriter
from tqdm import tqdm
import msgspec
import json
import re

http_errors = [
        "400 Bad Request", "401 Unauthorized", "402 Payment Required", "403 Forbidden", "404 Not Found",
        "405 Method Not Allowed", "406 Not Acceptable", "407 Proxy Authentication Required", "408 Request Timeout",
        "409 Conflict", "410 Gone", "411 Length Required", "412 Precondition Failed", "413 Payload Too Large",
        "414 URI Too Long", "415 Unsupported Media Type", "416 Range Not Satisfiable", "417 Expectation Failed",
        "418 I'm a teapot", "421 Misdirected Request", "422 Unprocessable Entity", "423 Locked", "424 Failed Dependency",
        "425 Too Early", "426 Upgrade Required", "428 Precondition Required", "429 Too Many Requests",
        "431 Request Header Fields Too Large", "451 Unavailable For Legal Reasons", "500 Internal Server Error",
        "501 Not Implemented", "502 Bad Gateway", "503 Service Unavailable", "504 Gateway Timeout",
        "505 HTTP Version Not Supported", "506 Variant Also Negotiates", "507 Insufficient Storage",
        "508 Loop Detected", "510 Not Extended", "511 Network Authentication Required"
    ]

rejected = [
    'Internal Server Error',
    '__NOEDITSECTION__',
    'enter your username and password',
    'forgotten your password',
    'cookies enabled',
    'enable JavaScript in your browser.',
    'The page cannot be displayed',
    'site or edit the error_page',
    'Request unsuccessful',
]

rejected.extend(http_errors)

def replace_multiple(input_string, pattern =r"\s{6,}", replace = '   '):
    return re.sub(pattern, replace, input_string)

def replace(string):
    string = replace_multiple(string.replace('…', '.'))
    string = replace_multiple(string, pattern = r"\.{6,}", replace = '...')
    return string

def reject(string):
    if any([r in string for r in rejected]):
        return True
    return False

In [3]:
files = [
    'lirik_lagu_dotnet_1000_end.jsonl',
    'lirik_lagu_dotnet_1_1000.jsonl'
]

with open('lirik_lagu_dotnet.dedup.jsonl', 'w') as fopen_l:
    for f in files:
        with open(f) as fopen:
            for l in fopen:
                l = json.loads(l)
                t = []
                for k, v in l.items():
                    if len(v) > 1:
                        v = replace(v)
                        t.append(f"{k.replace('_', ' ')}: {v.strip()}")

                t = '\n'.join(t)
                fopen_l.write(f'{json.dumps(t)}\n')

In [4]:
with open('upsi_ejournal_of_music.dedup.jsonl', 'w') as fopen_l:
    with open('journal_with_full_text.json') as fopen:
        d = json.load(fopen)
        
    for l in d:
        t = []
        for k, v in l.items():
            if isinstance(v, list):
                v = '\n'.join(v)
            if isinstance(v, str):
                v = replace(v)
            t.append(f"{k.replace('_', ' ')}: {v}".strip())

        t = '\n'.join(t)
        fopen_l.write(f'{json.dumps(t)}\n')

In [5]:
with open('foodie-my-food.dedup.jsonl', 'w') as fopen_l:
    with open('foodie-my-food.json') as fopen:
        d = json.load(fopen)
        
    for l in d:
        t = []
        for k, v in l.items():
            if isinstance(v, list):
                v = '\n'.join(v)
            if isinstance(v, str):
                v = replace(v)
            t.append(f"{k.replace('_', ' ')}: {v}".strip())

        t = '\n'.join(t)
        fopen_l.write(f'{json.dumps(t)}\n')

In [6]:
with open('malaysian_journal_of_analytical_science.dedup.jsonl', 'w') as fopen_l:
    with open('malaysian_journal_of_analytical_science.json') as fopen:
        d = json.load(fopen)
        
    for l in d:
        l = json.loads(l)
        t = []
        for k, v in l.items():
            if isinstance(v, list):
                v = '\n'.join(v)
            if isinstance(v, str):
                v = replace(v)
            t.append(f"{k.replace('_', ' ')}: {v}".strip())

        t = '\n'.join(t)
        fopen_l.write(f'{json.dumps(t)}\n')

In [7]:
with open('malaysian-journal-of-computing.dedup.jsonl', 'w') as fopen_l:
    with open('mjoc_pdf_text_extracted.json') as fopen:
        for l in fopen:
            l = json.loads(l)
            t = []
            for k, v in l.items():
                if isinstance(v, list):
                    v = '\n'.join(v)
                if isinstance(v, str):
                    v = replace(v)
                    if len(v) < 3:
                        continue
                t.append(f"{k.replace('_', ' ')}: {v}".strip())

            t = '\n'.join(t)
            fopen_l.write(f'{json.dumps(t)}\n')

In [8]:
with open('mishkat_al_masabih.dedup.jsonl', 'w') as fopen_l:
    with open('mishkat_al_masabih.jsonl') as fopen:
        for l in fopen:
            l = json.loads(l)
            t = []
            for k, v in l.items():
                if len(v) > 1:
                    v = replace(v)
                    t.append(f"{k.replace('_', ' ')}: {v.strip()}")
            
            t = '\n'.join(t)
            fopen_l.write(f'{json.dumps(t)}\n')

In [9]:
with open('umno-online.my.dedup.jsonl', 'w') as fopen_l:
    with open('article_data.json') as fopen:
        d = json.load(fopen)
    for d_ in d:
        t = []
        for k, v in d_.items():
            if isinstance(v, list):
                v = '\n'.join(v)
            v = replace(v)
            t.append(f"{k.replace('_', ' ')}: {v.strip()}")
        t = '\n'.join(t)
        fopen_l.write(f'{json.dumps(t)}\n')

In [10]:
with open('tamil-oneindia.dedup.jsonl', 'w') as fopen_l:
    with open('tamil-oneindia.json') as fopen:
        d = json.load(fopen)
    for d_ in d:
        t = []
        for k, v in d_.items():
            if isinstance(v, list):
                v = '\n'.join(v)
            v = replace(v)
            t.append(f"{k.replace('_', ' ')}: {v.strip()}")
        t = '\n'.join(t)
        fopen_l.write(f'{json.dumps(t)}\n')

In [11]:
import pandas as pd

df = pd.read_csv('train.csv').iloc[:,:-2]
df = df.to_dict('records')

with open('malaysia-elelong.dedup.jsonl', 'w') as fopen_l:
    for d_ in df:
        t = []
        for k, v in d_.items():
            if isinstance(v, list):
                v = '\n'.join(v)
            if isinstance(v, str):
                v = replace(v)
            t.append(f"{k.replace('_', ' ')}: {v}".strip())
        t = '\n'.join(t)
        fopen_l.write(f'{json.dumps(t)}\n')

In [12]:
with open('sejarahstpm.dedup.jsonl', 'w') as fopen_l:
    with open('sejarahstpm-text-extracted.json') as fopen:
        for l in fopen:
            l = json.loads(l)
            t = []
            for k, v in l.items():
                if isinstance(v, list):
                    v = '\n'.join(v)
                if isinstance(v, str):
                    v = replace(v)
                    if len(v) < 3:
                        continue
                t.append(f"{k.replace('_', ' ')}: {v}".strip())

            t = '\n'.join(t)
            fopen_l.write(f'{json.dumps(t)}\n')

In [13]:
with open('maljnutr.dedup.jsonl', 'w') as fopen_l:
    with open('maljnutr.json') as fopen:
        d = json.load(fopen)
        for l in d:
            t = []
            for k, v in l.items():
                if isinstance(v, list):
                    v = '\n'.join(v)
                if isinstance(v, str):
                    v = replace(v)
                    if len(v) < 3:
                        continue
                t.append(f"{k.replace('_', ' ')}: {v}".strip())

            t = '\n'.join(t)
            fopen_l.write(f'{json.dumps(t)}\n')

In [14]:
with open('muftiwp.gov.my.dedup.jsonl', 'w') as fopen_l:
    with open('muftiwp.gov.my.jsonl') as fopen:
        for l in fopen:
            l = json.loads(l)
            t = []
            for k, v in l.items():
                if isinstance(v, list):
                    v = '\n'.join(v)
                if isinstance(v, str):
                    v = replace(v)
                    if len(v) < 3:
                        continue
                t.append(f"{k.replace('_', ' ')}: {v}".strip())

            t = '\n'.join(t)
            fopen_l.write(f'{json.dumps(t)}\n')

In [15]:
files = [
    'jurnaldbp.jsonl',
    'mjpharm.org.jsonl',
    'myjgeosc.com.jsonl',
    'myjsustainagri.com.jsonl',
    'akademisains.gov.my.jsonl',
    'crossref-pdf.jsonl',
    'melayu-pdf.jsonl',
    'majcafe.com.jsonl',
    'myjms.mohe.gov.my.jsonl',
    'newera.edu.my.jsonl',
    'nutriweb.org.my.jsonl',
    'dermatology.org.my.jsonl',
    'e-mjm.org.jsonl',
    'pgm-my.org.jsonl',
    'msss.com.my.jsonl',
    'mps.org.my.jsonl',
    'morthoj.org.jsonl',
    'mjpath.org.my.jsonl',
]

for f in files[::-1]:
    
    new_f = f.replace('.jsonl', '.dedup.jsonl')
    with open(new_f, 'w') as fopen_l:
        with open(f) as fopen:
            for l in fopen:
                l = json.loads(l)
                if reject(l):
                    continue

                l = replace(l.strip())
                if len(l) < 3:
                    continue

                t = l
                fopen_l.write(f'{json.dumps(t)}\n')
    print(new_f, f)

mjpath.org.my.dedup.jsonl mjpath.org.my.jsonl
morthoj.org.dedup.jsonl morthoj.org.jsonl
mps.org.my.dedup.jsonl mps.org.my.jsonl
msss.com.my.dedup.jsonl msss.com.my.jsonl
pgm-my.org.dedup.jsonl pgm-my.org.jsonl
e-mjm.org.dedup.jsonl e-mjm.org.jsonl
dermatology.org.my.dedup.jsonl dermatology.org.my.jsonl
nutriweb.org.my.dedup.jsonl nutriweb.org.my.jsonl
newera.edu.my.dedup.jsonl newera.edu.my.jsonl
myjms.mohe.gov.my.dedup.jsonl myjms.mohe.gov.my.jsonl
majcafe.com.dedup.jsonl majcafe.com.jsonl
melayu-pdf.dedup.jsonl melayu-pdf.jsonl
crossref-pdf.dedup.jsonl crossref-pdf.jsonl
akademisains.gov.my.dedup.jsonl akademisains.gov.my.jsonl
myjsustainagri.com.dedup.jsonl myjsustainagri.com.jsonl
myjgeosc.com.dedup.jsonl myjgeosc.com.jsonl
mjpharm.org.dedup.jsonl mjpharm.org.jsonl
jurnaldbp.dedup.jsonl jurnaldbp.jsonl


In [16]:
from huggingface_hub import HfApi
api = HfApi()

In [18]:
from glob import glob

files = glob('*.dedup.jsonl')
for f in files:
    print(f)
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f,
        repo_id='malaysia-ai/dedup-text-dataset',
        repo_type='dataset',
    )

majcafe.com.dedup.jsonl


majcafe.com.dedup.jsonl:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

morthoj.org.dedup.jsonl


morthoj.org.dedup.jsonl:   0%|          | 0.00/25.6M [00:00<?, ?B/s]

e-mjm.org.dedup.jsonl


...

akademisains.gov.my.dedup.jsonl:   0%|          | 0.00/33.1M [00:00<?, ?B/s]

myjms.mohe.gov.my.dedup.jsonl


myjms.mohe.gov.my.dedup.jsonl:   0%|          | 0.00/451M [00:00<?, ?B/s]

foodie-my-food.dedup.jsonl


foodie-my-food.dedup.jsonl:   0%|          | 0.00/128k [00:00<?, ?B/s]